In [1]:
import emnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import methods as M
import sklearn
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
%matplotlib inline

In [5]:
n_train_classes = 40
n_test_classes = 7

In [9]:
train_images, train_labels, test_data = M.get_emnist(0, n_train_classes, n_test_classes, [1], False)
fewshot_images, fewshot_labels, val_images, val_labels = test_data[0]

In [10]:
def get_image_by_label(label):
    return train_images[np.random.choice(np.where(train_labels == label)[0], 1, False)[0]]

In [11]:
def get_train_data(size):
    targets = np.zeros((size,))
    targets[size // 2:] = 1
    pairs = [np.zeros((size, 28, 28)) for _ in range(2)]
    labels = np.unique(train_labels)
    for i in range(size):
        class1 = np.random.choice(labels, 1)[0]
        class2 = class1
        if i < size // 2:
            while class2 == class1:
                class2 = np.random.choice(labels, 1)[0]
        pairs[0][i] = get_image_by_label(class1)
        pairs[1][i] = get_image_by_label(class2)
    return pairs, targets

In [20]:
def get_siamese_net_and_encoder(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    encoder = Sequential()
    encoder.add(Conv2D(16, (3, 3), input_shape=input_shape, activation='relu', kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2, strides=(2, 2)))
    encoder.add(Dropout(0.25))
    
    encoder.add(Conv2D(32, (3, 3), kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2, strides=(2, 2)))
    encoder.add(Dropout(0.25))
    
    encoder.add(Flatten())
    
    encoder.add(Dense(32))
    
    left_emb = encoder(left_input)
    right_emb = encoder(right_input)
    
    L1_Layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_Dist = L1_Layer([left_emb,right_emb])
    OP = Dense(1, activation='sigmoid', kernel_regularizer='l2')(L1_Dist)
    
    siamese_net = Model(inputs=[left_input, right_input], outputs=OP)
    
    return siamese_net, encoder

In [24]:
num_iterations = 3000
batch_size = 10

_, w, h = train_images.shape

siamese_net, encoder = get_siamese_net_and_encoder((w, h, 1))

siamese_net.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

siamese_net.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 sequential_2 (Sequential)   (None, 32)                   30624     ['input_5[0][0]',             
                                                                     'input_6[0][0]']             
                                                                                                  
 lambda_2 (Lambda)           (None, 32)                   0         ['sequential_2[0][0]',  

In [25]:
for i in range(0, num_iterations):
    x, y = get_train_data(batch_size)
    siamese_net.fit(x, y)
    # if i % evaluateEvery == 0:
    #     print('Iteration:', i, '- loss:', loss[0], '- accuracy:', loss[1])

1/1 [==============================] - 0s 7ms/step - loss: 0.4385 - accuracy: 0.9000


In [23]:
pred = M.train_fewshot(encoder, 1, fewshot_images, fewshot_labels).predict(encoder(val_images))

print("Accuracy: ", np.sum(pred == val_labels)/len(val_labels))

Accuracy:  0.6096587863990949


In [21]:
M.test_NLE(train_images.reshape(-1, 28*28), train_labels, oneshot_images.reshape(-1, 28*28), oneshot_labels, validation_images.reshape(-1, 28*28), validation_labels, len(np.unique(train_labels)), 32, True, 1)

======= Nonlinear ae method: Training and evaluating ... =======
Learning background ...
Epoch 1/10
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0330
Epoch 2/10
3000/3000 [==============================] - 5s 2ms/step - loss: 0.0259
Epoch 3/10
3000/3000 [==============================] - 5s 2ms/step - loss: 0.0243
Epoch 4/10
3000/3000 [==============================] - 5s 2ms/step - loss: 0.0228
Epoch 5/10
3000/3000 [==============================] - 5s 2ms/step - loss: 0.0221
Epoch 6/10
3000/3000 [==============================] - 5s 2ms/step - loss: 0.0217
Epoch 7/10
3000/3000 [==============================] - 5s 2ms/step - loss: 0.0214
Epoch 8/10
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0211
Epoch 9/10
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0209
Epoch 10/10
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0208
Learning oneshot ...
Predicting ...
Accuracy:  0.5027690108973977
======= Nonlin

0.5027690108973977

In [23]:
M.test_CNE(train_images, train_labels, oneshot_images, oneshot_labels, validation_images, validation_labels, len(np.unique(train_labels)), 32, True, 1, 28, 28, 1, 28*28)

======= CNN ae method: Training and evaluating ... =======
Learning background ...
Epoch 1/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0210
Epoch 2/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.0130
Epoch 3/10
3000/3000 [==============================] - 19s 6ms/step - loss: 0.0117
Epoch 4/10
3000/3000 [==============================] - 19s 6ms/step - loss: 0.0111
Epoch 5/10
3000/3000 [==============================] - 19s 6ms/step - loss: 0.0107
Epoch 6/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0104
Epoch 7/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0102
Epoch 8/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0101
Epoch 9/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0100
Epoch 10/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.0099
Learning oneshot ...
Predicting ...
Accuracy:  0.4670398380277497
======= CN

0.4670398380277497